In [79]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline

In [171]:
"""
df = pd.read_excel("./custom-data/population_okinawa.xlsx",
                   skiprows=10, usecols="B,D",
                   )

df.columns = ["place", "population"]
df["population"] = df["population"].str.replace(",", "").astype(int)
df
"""

population = pd.read_excel("./custom-data/population//2018-a.xls", header=2).dropna(how="any", axis=0)
population.columns = population.columns.str.replace("Unnamed: 1", "市区町村名").str.replace("\n", "-")
population["市区町村名"] = population["市区町村名"].apply(lambda x: x.replace("\u3000", ""))
population

,市 区 町 村,市区町村名,人口総数,15歳未満-人口,15～64歳-人口,65歳以上-人口,外国人人口,人口集中-地区人口,出生数,死亡数,...,世帯数,一般世帯数,核家族-世帯数,単独世帯数,65歳以上の世帯員の-いる核家族世帯数,高齢夫婦-世帯数,高齢単身-世帯数,婚姻件数,離婚件数,市区-町村-ｺｰﾄﾞ
4,1.0,北海道,5381733,608296,3190804,1558387,21676,4047281,35125,61906,...,2444810,2438206,1363818,909106,558618,327509,319408,24636,10476,1.0
5,1100.0,札幌市,1952356,221013,1235516,483534,8820,1899081,14021,18504,...,921837,920415,493644,375242,178246,98660,104650,10495,4096,1100.0
6,1101.0,中央区,237627,23997,160339,50536,1891,235356,1722,1995,...,132006,131819,52695,73164,16633,9542,14403,1723,572,1101.0
7,1102.0,北区,285321,32916,178950,68960,2023,275495,1848,2558,...,133662,133477,70121,55247,25763,14221,14688,1316,571,1102.0
8,1103.0,東区,261912,30716,168302,61448,1121,255507,2204,2449,...,124425,124251,65606,51879,22696,12156,14050,1608,608,1103.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1963,47361.0,久米島町,7755,1218,4475,2052,34,...,58,109,...,3365,3360,1798,1225,775,331,425,48,18,47361.0
1964,47362.0,八重瀬町,29066,5315,18012,5610,60,8212,405,230,...,9625,9598,6538,1915,2000,744,708,177,58,47362.0
1965,47375.0,多良間村,1194,208,671,315,14,...,12,15,...,470,470,268,142,118,49,51,3,1,47375.0
1966,47381.0,竹富町,3998,680,2469,798,38,...,57,48,...,2122,2121,832,1156,281,127,210,39,12,47381.0


In [172]:
population["市区町村名"].values

array(['北海道', '札幌市', '  中央区', ..., '多良間村', '竹富町', '与那国町'], dtype=object)

In [173]:
last = ""
prefecture = []
for place in population["市区町村名"].values:
    if place[-1] in ["都", "道", "府", "県"]:
        last = place
        prefecture.append(last)
    else:
        prefecture.append(last)

custom_population = population.copy()
custom_population["都道府県"] = prefecture
custom_population = custom_population.drop(custom_population[custom_population["市区町村名"] == custom_population["都道府県"]].index, axis=0)

custom_population = custom_population.drop(['婚姻件数', '離婚件数', '市区-町村-ｺｰﾄﾞ',
                                            '65歳以上の世帯員の-いる核家族世帯数', '核家族-世帯数',
                                            '高齢夫婦-世帯数', '高齢単身-世帯数', '人口集中-地区人口',
                                            "世帯数", "一般世帯数", "単独世帯数", "市 区 町 村"], axis=1)

custom_population = custom_population.set_index("都道府県")
custom_population = custom_population.reset_index()

custom_population = custom_population.astype({
                                            "都道府県": str,
                                            "市区町村名": str,
                                            "人口総数": int,
                                            "15歳未満-人口": int,
                                            "15～64歳-人口": int,
                                            "65歳以上-人口": int,
                                            "外国人人口": int,
                                            "出生数": int,
                                            "死亡数": int,
                                            "転入者数": int,
                                            "転出者数": int,
                                            "昼間人口": int,
                                            })
custom_population

,都道府県,市区町村名,人口総数,15歳未満-人口,15～64歳-人口,65歳以上-人口,外国人人口,出生数,死亡数,転入者数,転出者数,昼間人口
0,北海道,札幌市,1952356,221013,1235516,483534,8820,14021,18504,118336,109199,1959740
1,北海道,中央区,237627,23997,160339,50536,1891,1722,1995,20274,18490,386602
2,北海道,北区,285321,32916,178950,68960,2023,1848,2558,15580,13752,273734
3,北海道,東区,261912,30716,168302,61448,1121,2204,2449,15466,13889,247799
4,北海道,白石区,209584,23590,136879,47879,595,1849,2027,14188,13184,206604
...,...,...,...,...,...,...,...,...,...,...,...,...
1912,沖縄県,久米島町,7755,1218,4475,2052,34,58,109,428,516,7790
1913,沖縄県,八重瀬町,29066,5315,18012,5610,60,405,230,1563,1348,24992
1914,沖縄県,多良間村,1194,208,671,315,14,12,15,59,65,1180
1915,沖縄県,竹富町,3998,680,2469,798,38,57,48,517,502,4197


In [174]:
custom_population.to_excel("./custom-data/population/custom_population.xlsx", index=False)

In [175]:
wifi = pd.read_csv("./custom-data/wifi/470007_public_wireless_lan.csv")
wifi

,都道府県コード又は市区町村コード,NO,都道府県名,市区町村名,名称,名称_カナ,名称_英語,住所,方書,緯度,経度,設置者,電話番号,内線番号,SSID,提供エリア,URL,備考
0,472018,NaN,沖縄県,那覇市,今酒屋米丸,NaN,NaN,那覇市 港町2-3-8,NaN,26.243268,127.680657,沖縄県,NaN,NaN,Be.Okinawa_Free_Wi-Fi,NaN,NaN,NaN
1,472018,NaN,沖縄県,那覇市,沖縄鮮魚卸琉通協同組合,NaN,NaN,那覇市 港町1-1-18,NaN,26.251666,127.720086,沖縄県,NaN,NaN,Be.Okinawa_Free_Wi-Fi,NaN,NaN,NaN
2,472018,NaN,沖縄県,那覇市,居酒屋 イカリ亭,NaN,NaN,那覇市 曙1-20-25,NaN,26.238608,127.683046,沖縄県,NaN,NaN,Be.Okinawa_Free_Wi-Fi,NaN,NaN,NaN
3,472018,NaN,沖縄県,那覇市,M’sニューポート,NaN,NaN,那覇市 曙2-1-1,NaN,26.240959,127.689195,沖縄県,NaN,NaN,Be.Okinawa_Free_Wi-Fi,NaN,NaN,NaN
4,472018,NaN,沖縄県,那覇市,バリバリ,NaN,NaN,那覇市 曙1-13-17,1F,26.236705,127.681789,沖縄県,NaN,NaN,Be.Okinawa_Free_Wi-Fi,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2872,472077,NaN,沖縄県,石垣市,石垣SSグループ 白保サービスステーション,NaN,NaN,石垣市 白保764,NaN,24.338209,124.160188,沖縄県,NaN,NaN,Be.Okinawa_Free_Wi-Fi,NaN,NaN,NaN
2873,472077,NaN,沖縄県,石垣市,新石垣空港 国際線,NaN,NaN,石垣市 白保,NaN,24.396430,124.244996,沖縄県,NaN,NaN,Be.Okinawa_Free_Wi-Fi,NaN,NaN,NaN
2874,472077,NaN,沖縄県,石垣市,石垣SSグループ 磯辺サービスステーション,NaN,NaN,石垣市 宮良1054-44,NaN,24.358280,124.208498,沖縄県,NaN,NaN,Be.Okinawa_Free_Wi-Fi,NaN,NaN,NaN
2875,472077,NaN,沖縄県,石垣市,石垣SSグループ 川平サービスステーション,NaN,NaN,石垣市 川平853-1,NaN,24.463770,124.141494,沖縄県,NaN,NaN,Be.Okinawa_Free_Wi-Fi,NaN,NaN,NaN


In [176]:
custom_wifi = wifi.copy()[["都道府県名", "市区町村名", "名称", "住所"]]

custom_wifi["市区町村名"] = custom_wifi["市区町村名"].str.strip()
custom_wifi

,都道府県名,市区町村名,名称,住所
0,沖縄県,那覇市,今酒屋米丸,那覇市 港町2-3-8
1,沖縄県,那覇市,沖縄鮮魚卸琉通協同組合,那覇市 港町1-1-18
2,沖縄県,那覇市,居酒屋 イカリ亭,那覇市 曙1-20-25
3,沖縄県,那覇市,M’sニューポート,那覇市 曙2-1-1
4,沖縄県,那覇市,バリバリ,那覇市 曙1-13-17
...,...,...,...,...
2872,沖縄県,石垣市,石垣SSグループ 白保サービスステーション,石垣市 白保764
2873,沖縄県,石垣市,新石垣空港 国際線,石垣市 白保
2874,沖縄県,石垣市,石垣SSグループ 磯辺サービスステーション,石垣市 宮良1054-44
2875,沖縄県,石垣市,石垣SSグループ 川平サービスステーション,石垣市 川平853-1


In [177]:
custom_wifi.to_excel("./custom-data/wifi/custom_470007_public_wireless_lan.xlsx", index=False)

In [178]:
economy = pd.read_excel("./custom-data/economy/2018-c.xls", header=2)
economy = economy.dropna(how="any", axis=0)

economy.columns = economy.columns.str.replace("Unnamed: 1", "市区町村名").str.replace("\n", "-")
economy["市区町村名"] = economy["市区町村名"].apply(lambda x: x.replace("\u3000", ""))

last = ""
prefecture = []
for place in economy["市区町村名"].values:
    if place[-1] in ["都", "道", "府", "県"]:
        last = place
        prefecture.append(last)
    else:
        prefecture.append(last)

custom_economy = economy.copy()
custom_economy["都道府県"] = prefecture
custom_economy = custom_economy.drop(custom_economy[custom_economy["市区町村名"] == custom_economy["都道府県"]].index, axis=0)

custom_economy = custom_economy.set_index("都道府県")
custom_economy = custom_economy.reset_index()


custom_economy = custom_economy[[
        '都道府県', '市区町村名', '事業所数',
        '第２次産業事業所数', '第３次産業事業所数',
       ]]

custom_economy["市区町村名"] = custom_economy["市区町村名"].str.strip()

custom_economy

,都道府県,市区町村名,事業所数,第２次産業事業所数,第３次産業事業所数
0,北海道,札幌市,76604,9321,67178
1,北海道,中央区,23138,1132,21971
2,北海道,北区,9331,1440,7883
3,北海道,東区,8964,1726,7232
4,北海道,白石区,8276,1432,6837
...,...,...,...,...,...
1912,沖縄県,久米島町,604,147,440
1913,沖縄県,八重瀬町,790,133,641
1914,沖縄県,多良間村,87,14,71
1915,沖縄県,竹富町,489,37,441


In [179]:
custom_economy.to_excel("./custom-data/economy/custom_2018-c.xlsx", index=False)

In [180]:
store = pd.read_excel("./custom-data/store/2018-h.xls", header=2)

store = store.dropna(how="any", axis=0)

store.columns = store.columns.str.replace("Unnamed: 1", "市区町村名").str.replace("\n", "-")
store["市区町村名"] = store["市区町村名"].apply(lambda x: x.replace("\u3000", ""))

last = ""
prefecture = []
for place in store["市区町村名"].values:
    if place[-1] in ["都", "道", "府", "県"]:
        last = place
        prefecture.append(last)
    else:
        prefecture.append(last)

custom_store = store.copy()
custom_store["都道府県"] = prefecture
custom_store = custom_store.drop(custom_store[custom_store["市区町村名"] == custom_store["都道府県"]].index, axis=0)

custom_store = custom_store.set_index("都道府県")
custom_store = custom_store.reset_index()

custom_store = custom_store[[
        '都道府県', '市区町村名',
        '小売店数', '飲食店数',
        ]]

custom_store["市区町村名"] = custom_store["市区町村名"].str.strip()

custom_store

,都道府県,市区町村名,小売店数,飲食店数
0,北海道,札幌市,11498,9124
1,北海道,中央区,3262,4327
2,北海道,北区,1436,972
3,北海道,東区,1359,631
4,北海道,白石区,1095,620
...,...,...,...,...
1912,沖縄県,久米島町,111,58
1913,沖縄県,八重瀬町,155,53
1914,沖縄県,多良間村,13,5
1915,沖縄県,竹富町,58,57


In [181]:
custom_store.to_excel("./custom-data/store/custom_2018-h.xlsx", index=False)